In [1]:
import pyomo.environ as pyo 
from pyomo.environ import *
from pyomo.opt import SolverFactory
import math 
import pandas as pd
import os
os.chdir('C:\\Users\\omkarp\\Downloads\\Optimize\\')

# Project: Planning of electricity production and transmission

1. Formulate a model minimizing the cost of production of active power

**Parameters for the model:**

- Nodes (k,l) -ranging from 1 to 11 over transmission grid network
- Shunt admittance on edges:   $b_{kl}$ and $g_{kl}$ between nodes
- Generator Max Capacity (pu):  $G_{ki}$ with i for Generator number ranging from 1 to 9, and k for node location
- Generator Energy production cost (SEK/pu):  $GEcost_{ki}$ with i for Gen.no. ranging from 1 to 9, and k for node location
- Consumer Demand for Active Power: $C_{li}$ with i for Consumer number ranging from 1 to 7, and l for node location


## Get data

In [2]:
shunt_data = pd.read_excel('Project.xlsx', sheet_name='shunt')
shunt_data[['k', 'l']] = shunt_data['Edge(kl)'].str.split('_', expand=True)
shunt_data['k'] = shunt_data['k'].astype(int)
shunt_data['l'] = shunt_data['l'].astype(int)
shunt_data.head()

,Edge(kl),bkl,gkl,k,l
0,1_2,-20.1,4.12,1,2
1,1_11,-22.3,5.67,1,11
2,2_3,-16.8,2.41,2,3
3,2_11,-17.2,2.78,2,11
4,3_4,-11.7,1.98,3,4


In [3]:
generator_data = pd.read_excel('Project.xlsx', sheet_name='generator')
consumer_data = pd.read_excel('Project.xlsx', sheet_name='consumer')
print([consumer_data.Demand_active_power[i] for i in consumer_data.index])
generator_data.head()

[0.1, 0.19, 0.11, 0.09, 0.21, 0.05, 0.04]


,Generator,node,MaxCapacity,Energy_prod_cost
0,G1,2,0.02,175
1,G2,2,0.15,100
2,G3,2,0.08,150
3,G4,3,0.07,150
4,G5,4,0.04,300


In [4]:
powerplant = pd.read_excel('Project.xlsx', sheet_name='PowerPlant') # not used
consumer_data.head()

,Consumer,node,Demand_active_power
0,C1,1,0.10
1,C2,4,0.19
2,C3,6,0.11
3,C4,8,0.09
4,C5,9,0.21


## **MappIng nodes - edges to generators and consumers and power-plants for the model:**

In [5]:
node_to_edges = {}
node_to_generators = {}
node_to_consumers = {}

connections = {
    1: [2, 11],
    2: [1, 3, 11],
    3: [2, 4, 9],
    4: [3, 5],
    5: [4, 6, 8],
    6: [5, 7],
    7: [6, 8, 9],
    8: [5, 7, 9],
    9: [10, 3, 8, 7],
    10: [9, 11],
    11: [1, 2, 10]
}
for node, connected_nodes in connections.items():
    node_to_edges[node] = connected_nodes
    
node_to_consumers[1] = ['C1']                 # Node 1
node_to_generators[2] = ['G1', 'G2', 'G3']    # Node 2
node_to_generators[3] = ['G4']                # Node 3
node_to_generators[4] = ['G5', 'C2']          # Node 4
node_to_consumers[5] = ['G6']
node_to_generators[6] = ['C3']                # Node 6
node_to_consumers[7] = ['G7']                 # Node 7
node_to_generators[8] = ['C4']                # Node 8
node_to_consumers[9] = ['C5','G8', 'G9']      # Node 9
node_to_generators[10] = ['C6']                # Node 10
node_to_consumers[11] = ['C7']                # Node 11

In [6]:
node = 9
if node in node_to_edges:
    edges = node_to_edges[node]
    print(f"Edges for Node {node}: {edges}")

if node in node_to_generators:
    generators = node_to_generators[node]
    print(f"Generators for Node {node}: {generators}")

if node in node_to_consumers:
    consumers = node_to_consumers[node]
    print(f"Consumers for Node {node}: {consumers}")

Edges for Node 9: [10, 3, 8, 7]
Consumers for Node 9: ['C5', 'G8', 'G9']


In [7]:
model = pyo.ConcreteModel()

### Setting model parameters

In [8]:
# Sets
node_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
model.NODE = pyo.Set(initialize=node_indices)

generator_indices = {name: idx for idx, name in enumerate(generator_data['Generator'])} # initialize as integer indices
model.GENERATOR = pyo.Set(initialize=generator_indices.values())

consumer_indices = {name: idx for idx, name in enumerate(['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7'])}
model.CONSUMERINDEX = pyo.Set(initialize=consumer_indices.values())

In [9]:
# Parameters: Generator
max_capacity_values = {}
energy_cost_values = {}
generator_location = {}

for generator in generator_data.iterrows():
    generator_name = generator[1]['Generator']
    generator_idx = generator_indices[generator_name]  # integer index
    node_location = generator[1]['node']
    max_capacity = generator[1]['MaxCapacity']
    max_capacity_values[generator_idx] = max_capacity
    generator_location[generator_idx] = node_location
    
for generator in generator_data.iterrows():
    generator_name = generator[1]['Generator']
    generator_idx = generator_indices[generator_name] 
    energy_cost = generator[1]['Energy_prod_cost']
    energy_cost_values[generator_idx] = energy_cost
    
model.max_capacity = pyo.Param(model.GENERATOR, initialize=max_capacity_values)
model.cost = pyo.Param(model.GENERATOR, initialize=energy_cost_values)

generator_location_values = {generator_idx: node_location for generator_idx, node_location in generator_location.items()}
model.generator_location = pyo.Param(model.GENERATOR, initialize=generator_location_values)

for generator in model.GENERATOR:
    max_capacity_value = model.max_capacity[generator]
    print(f'Max Capacity for Generator {generator+1} = {max_capacity_value} pu')

Max Capacity for Generator 1 = 0.02 pu
Max Capacity for Generator 2 = 0.15 pu
Max Capacity for Generator 3 = 0.08 pu
Max Capacity for Generator 4 = 0.07 pu
Max Capacity for Generator 5 = 0.04 pu
Max Capacity for Generator 6 = 0.17 pu
Max Capacity for Generator 7 = 0.17 pu
Max Capacity for Generator 8 = 0.26 pu
Max Capacity for Generator 9 = 0.05 pu


In [10]:
# Parameters: Consumers
demand_values = {}
consumer_location = {}

for consumer in consumer_data.iterrows():
    consumer_name = consumer[1]['Consumer']
    node_location = consumer[1]['node']
    demand = consumer[1]['Demand_active_power']
    consumer_idx = consumer_indices[consumer_name]
    demand_values[consumer_idx] = demand
    consumer_location[consumer_idx] = node_location
    
model.demand = pyo.Param(model.CONSUMERINDEX, initialize=demand_values)
model.consumer_location = pyo.Param(model.CONSUMERINDEX, initialize=consumer_location)

for consumer in model.CONSUMERINDEX:
    demand_value = model.demand[consumer]
    print(f'Demand active power for Consumer {consumer+1} = {demand_value} pu')

Demand active power for Consumer 1 = 0.1 pu
Demand active power for Consumer 2 = 0.19 pu
Demand active power for Consumer 3 = 0.11 pu
Demand active power for Consumer 4 = 0.09 pu
Demand active power for Consumer 5 = 0.21 pu
Demand active power for Consumer 6 = 0.05 pu
Demand active power for Consumer 7 = 0.04 pu


In [11]:
# Parameters for Shunt Admittance
bkl_values = {}
gkl_values = {}

for k, connected_nodes in node_to_edges.items():
    for l in connected_nodes:
        if k != l:
            matching_rows = shunt_data[(shunt_data['k'] == k) & (shunt_data['l'] == l)]
            if not matching_rows.empty:
                gkl = matching_rows['gkl'].values[0]
                bkl = matching_rows['bkl'].values[0]
                gkl_values[(k, l)] = gkl
                gkl_values[(l, k)] = gkl
                bkl_values[(k, l)] = bkl
                bkl_values[(l, k)] = bkl

model.bkl = Param(model.NODE, model.NODE, initialize=bkl_values)
model.gkl = Param(model.NODE, model.NODE, initialize=gkl_values)

In [12]:
for k, connected_nodes in node_to_edges.items():
    for l in connected_nodes:
        if (k, l) in model.gkl:
            gkl_value = model.gkl[k, l]
            print(f'gkl[{k}, {l}] = {gkl_value}')
            break

gkl[1, 2] = 4.12
gkl[2, 1] = 4.12
gkl[3, 2] = 2.41
gkl[4, 3] = 1.98
gkl[5, 4] = 1.59
gkl[6, 5] = 1.71
gkl[7, 6] = 1.11
gkl[8, 5] = 1.26
gkl[9, 10] = 2.14
gkl[10, 9] = 2.14
gkl[11, 1] = 5.67


## Set variables

**Variables for the model:**

- Voltage Amplitude ($v_{k}$ for each node k) between 0.98 and 1.00
- Voltage angles ($\theta_{k}$ for each node k) between pi and -pi (radians)
- Power flows for active ($p.var_{k}$ for each node k) and for reactive ($q.var_{k}$ for each node k)
- Generated active and reactive power by each generator

In [13]:
model.v = pyo.Var(model.NODE, within=NonNegativeReals, bounds=(0.98, 1.02))  # voltage amplitudes
model.theta = pyo.Var(model.NODE, bounds=(-math.pi, math.pi))  # voltage angle (in radians)

#active and reactive power flow for each edge (between nodes k and l)
model.p_var = pyo.Var(model.NODE, model.NODE, within=pyo.Reals)  # active
model.q_var = pyo.Var(model.NODE, model.NODE, within=pyo.Reals)

# bounds for active and reactive power
for k in model.NODE:
    for l in model.NODE:
        if k != l:
            model.p_var[k, l].setlb(0.0)
            model.q_var[k, l].setlb(-0.003 * max_capacity_value)
            model.q_var[k, l].setub(0.003 * max_capacity_value)

In [14]:
# For each generator
model.active_power = pyo.Var(model.GENERATOR, within=pyo.NonNegativeReals, bounds=(0, model.max_capacity))

def reactive_power_bounds(model, generator):
    max_capacity = model.max_capacity[generator]
    return (-0.003 * max_capacity, 0.003 * max_capacity)
model.reactive_power = pyo.Var(model.GENERATOR, bounds=reactive_power_bounds)

## Set Constraints

### **Amount of active power flow (p_var) between nodes**

$$\begin{aligned}
p_{kℓ} = (v_k)^2 * g_{kℓ} − v_k * v_ℓ * g_{kℓ} * cos(θ_k − θ_ℓ) − v_k * v_ℓ * b_{kℓ} * sin(θ_k − θ_ℓ)
\end{aligned}$$

In [15]:
def active_power_flow_rule(model, k, l):
    if (k, l) in model.gkl:
        return model.p_var[k, l] == (model.v[k] ** 2) * model.gkl[k, l] - model.v[k] * model.v[l] * model.gkl[k, l] * pyo.cos(model.theta[k] - model.theta[l]) - model.v[k] * model.v[l] * model.bkl[k, l] * pyo.sin(model.theta[k] - model.theta[l])
    else:
        return pyo.Constraint.Skip # ifedge (k, l) does not exist

model.active_power_flow_constraint = pyo.Constraint(model.NODE, model.NODE, rule=active_power_flow_rule)

### **Amount of reactive power flow (q_var) between nodes**

$$\begin{aligned}
q_{kℓ} = -(v_k)^2 * b_{kℓ} + v_k * v_ℓ * b_{kℓ} * cos(θ_k − θ_ℓ) − v_k * v_ℓ * g_{kℓ} * sin(θ_k − θ_ℓ)
\end{aligned}$$

In [16]:
def reactive_power_flow_rule(model, k, l):
    if (k, l) in model.bkl:
        return model.q_var[k, l] == - (model.v[k] ** 2) * model.bkl[k, l] + model.v[k] * model.v[l] * model.bkl[k, l] * pyo.cos(model.theta[k] - model.theta[l]) - model.v[k] * model.v[l] * model.gkl[k, l] * pyo.sin(model.theta[k] - model.theta[l])
    else:
        return pyo.Constraint.Skip

model.reactive_power_flow_constraint = pyo.Constraint(model.NODE, model.NODE, rule=reactive_power_flow_rule)

### **Power Balance: generation and consumption (at each node)**

$$\begin{aligned}
\text{PowerGeneration}=\text{PowerConsumption....(at each node)}  
\end{aligned}$$

In [17]:
def active_power_balance_rule(model, k):    #  total active power injection at node k
    total_active_power_injection = (
        sum(model.p_var[k, l] for l in node_to_edges[k]) - 
        sum(model.p_var[l, k] for l in node_to_edges[k]) +
        sum(model.max_capacity[g] * model.active_power[g]
            for g in model.GENERATOR if model.generator_location[g] == k)
    )
    if k in model.consumer_location.values(): # total active power demand at node k
        total_active_power_demand = sum(model.demand[c] for c in model.CONSUMERINDEX if model.consumer_location[c] == k)
    else:
        total_active_power_demand = 0.0
    return total_active_power_injection == total_active_power_demand     # active power balance at node k

model.active_power_balance_constraint = pyo.Constraint([k for k in model.NODE], rule=active_power_balance_rule)

In [18]:
generator= 0
print(model.max_capacity[generator])
for generator in model.max_capacity: print(generator)

0.02
0
1
2
3
4
5
6
7
8


In [19]:
# total reactive power injection at node k
def reactive_power_balance_rule(model, k):
    total_reactive_power_injection = (
        sum(model.q_var[k, l] for l in node_to_edges[k]) - 
        sum(model.q_var[l, k] for l in node_to_edges[k]) +
        sum((model.v[k]**2) * (model.bkl[k, l] - model.gkl[k, l] * pyo.cos(model.theta[k] - model.theta[l]))
            for l in node_to_edges[k])
    )
    return total_reactive_power_injection == 0.0  #  reactive power balance at node k

model.reactive_power_balance_constraint = pyo.Constraint([k for k in model.NODE], rule=reactive_power_balance_rule)

## Set Objective Function

**To minimize the cost of total active power generated for consumption:**

$$\begin{aligned}
Minimize\ (Cost_{activePower})
\end{aligned}$$


In [20]:
def objective_rule(model):
    objective_value = sum(
        model.cost[g] * model.active_power[g]
        for g in model.GENERATOR
    )
    return objective_value

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

In [21]:
solver = SolverFactory('ipopt', executable='C:\\ipopt\\bin\\ipopt.exe')
solver.solve(model)

TypeError: Cannot compute the value of an indexed Param (max_capacity)

## Results

In [ ]:
active_power = {k: pyo.value(model.active_power[k]) for i in model.GENERATOR}
reactive_power = {k: pyo.value(model.reactive_power[k]) for i in model.GENERATOR}
voltage_amplitudes = {k: pyo.value(model.v[k]) for i in model.NODE}
voltage_angles = {k: pyo.value(model.theta[k]) for i in model.NODE}
total_cost = pyo.value(model.objective)

In [ ]:
print("Active Power Production:", active_power)
print("Reactive Power Production:", reactive_power)
print("Voltage Amplitudes:", voltage_amplitudes)
print("Voltage Angles:", voltage_angles)
print("Total Cost of Production:", total_cost)

In [ ]:
model.pprint()